In [1]:
import cv2
import mediapipe as mp
import numpy as np
import screen_brightness_control as sbc

# Initialize MediaPipe Hands and Drawing Utilities
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)
hands = mp_hands.Hands()

try:
    while True:
        success, frame = cap.read()
        if not success:
            break

        # Flip the frame horizontally for a later selfie-view display
        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame and find hands
        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw landmarks
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Extract the coordinates of the thumb (landmark 4) and index finger (landmark 8)
                x_thumb, y_thumb = int(hand_landmarks.landmark[4].x * frame.shape[1]), int(hand_landmarks.landmark[4].y * frame.shape[0])
                x_index, y_index = int(hand_landmarks.landmark[8].x * frame.shape[1]), int(hand_landmarks.landmark[8].y * frame.shape[0])

                # Draw circles on thumb and index finger tips
                cv2.circle(frame, (x_thumb, y_thumb), 10, (0, 0, 255), -1)
                cv2.circle(frame, (x_index, y_index), 10, (0, 0, 255), -1)

                # Calculate the distance between the thumb and index finger tips
                distance = np.sqrt((x_index - x_thumb) ** 2 + (y_index - y_thumb) ** 2)

                # Adjust the screen brightness based on the distance
                brightness = np.clip(int(distance), 0, 100)
                sbc.set_brightness(brightness)

                # Display the current brightness on the frame
                cv2.putText(frame, f'Brightness: {brightness}%', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        # Display the frame
        cv2.imshow('Hand Gesture Recognition for Brightness Control', frame)

        if cv2.waitKey(1) & 0xFF == 27:  # Exit on pressing ESC
            break

except KeyboardInterrupt:
    pass

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


C:\Users\viji0\anaconda3\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
